# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 

"""
for i in range(len(twits['data'][:])):
    print (i)

#len(twits['data'][:].keys())

from collections import Counter

counter = Counter(twits(['message_body']))

counter

def thebest():
  entries = [d['data'] for d in list if d['data']]
  return len(entries), sum(entries)
"""

len(twits['data'])

1548010

In [4]:
twits.keys()

dict_keys(['data'])

In [5]:
len(twits.keys())

1

In [6]:
len(twits['data'][0:])

1548010

### Split Message Body and Sentiment Score

In [7]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [8]:
messages

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option straddle near the supporting

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [9]:
text = (twits['data'][:10])
#text = text.lower()
#print(text.head())
print(text)

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

In [10]:
text = str(text)
text = text.lower()
print(text)

[{'message_body': '$fitb great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01t00:00:09z'}, {'message_body': '@stocktwits $msft', 'sentiment': 1, 'timestamp': '2018-07-01t00:00:42z'}, {'message_body': '#staanalystalert for $tdg : jefferies maintains with a rating of hold setting target price at usd 350.00. our own verdict is buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01t00:01:24z'}, {'message_body': '$amd i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', 'sentiment': 1, 'timestamp': '2018-07-01t00:01:47z'}, {'message_body': '$amd reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01t00:02:13z'}, {'message_body': '$aapl why the drop? i warren buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01t00:03:10z'}, {'message_body': '$ba bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=ba', 'sentiment': -2, 'timestamp': '2018-07-01t

In [11]:
#text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text)
#text =  re.sub(r'\<http.+?\>', ' ', text, flags=re.S)
regex1 = re.compile(r"http+\:?/\/.*\'")
                   
matches1 = regex1.finditer(text)

for match in matches1:
    print (match)

<_sre.SRE_Match object; span=(351, 1469), match="http://www.stocktargetadvisor.com/toprating', 'se>


In [12]:
#match = re.search(r'([\w\.-]+)@([\w\.-]+)', text)

match = re.search(r'([\w\.-]+)://([\w\.-]+)', text)

if match:
  print(match.group()) # The whole matched text
  print(match.group(1)) # The username (group 1)
  print(match.group(2)) # The host (group 2)

http://www.stocktargetadvisor.com
http
www.stocktargetadvisor.com


In [13]:
regex = re.compile(r'([\w\.-]+)://([\w\.-]+)')

matches = regex.finditer(text)

for match in matches:
    print (match)


<_sre.SRE_Match object; span=(351, 384), match='http://www.stocktargetadvisor.com'>
<_sre.SRE_Match object; span=(927, 950), match='https://dividendbot.com'>


In [14]:
#regex2 = re.compile(r'https.+?/\/.*[\r\n]*')
regex2 = re.compile(r'https.+?/\/.*')
                   
matches2 = regex2.finditer(text)

for match in matches2:
    print (match)

<_sre.SRE_Match object; span=(927, 1471), match="https://dividendbot.com?s=ba', 'sentiment': -2, '>


In [15]:
#remove urls it worked!!!

remove = re.sub(r'([\w\.-]+)://([\w\.-]+)', " ", text)

remove

"[{'message_body': '$fitb great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01t00:00:09z'}, {'message_body': '@stocktwits $msft', 'sentiment': 1, 'timestamp': '2018-07-01t00:00:42z'}, {'message_body': '#staanalystalert for $tdg : jefferies maintains with a rating of hold setting target price at usd 350.00. our own verdict is buy   /toprating', 'sentiment': 2, 'timestamp': '2018-07-01t00:01:24z'}, {'message_body': '$amd i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', 'sentiment': 1, 'timestamp': '2018-07-01t00:01:47z'}, {'message_body': '$amd reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01t00:02:13z'}, {'message_body': '$aapl why the drop? i warren buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01t00:03:10z'}, {'message_body': '$ba bears have 1 reason on 06-29 to pay more attention  ?s=ba', 'sentiment': -2, 'timestamp': '2018-07-01t00:04:09z'}, {'message_body': '$bac ok good we&#39;re

In [16]:
#find stock tickers

#regex = re.compile(r'([\w\.-]+)://([\w\.-]+)')
regex = re.compile(r'\$([\w\.-]+)')
#regex = re.compile(r'([\w\.-]+)\$([\w\.-]+)')
matches = regex.finditer(text)

for match in matches:
    print (match)

<_sre.SRE_Match object; span=(19, 24), match='$fitb'>
<_sre.SRE_Match object; span=(141, 146), match='$msft'>
<_sre.SRE_Match object; span=(242, 246), match='$tdg'>
<_sre.SRE_Match object; span=(469, 473), match='$amd'>
<_sre.SRE_Match object; span=(641, 645), match='$amd'>
<_sre.SRE_Match object; span=(737, 742), match='$aapl'>
<_sre.SRE_Match object; span=(872, 875), match='$ba'>
<_sre.SRE_Match object; span=(1031, 1035), match='$bac'>
<_sre.SRE_Match object; span=(1172, 1177), match='$amat'>
<_sre.SRE_Match object; span=(1300, 1304), match='$gme'>


In [17]:
#remove tickers
remove_tick = re.sub(r'\$([\w\.-]+)', " ", text)

remove_tick

"[{'message_body': '  great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01t00:00:09z'}, {'message_body': '@stocktwits  ', 'sentiment': 1, 'timestamp': '2018-07-01t00:00:42z'}, {'message_body': '#staanalystalert for   : jefferies maintains with a rating of hold setting target price at usd 350.00. our own verdict is buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01t00:01:24z'}, {'message_body': '  i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', 'sentiment': 1, 'timestamp': '2018-07-01t00:01:47z'}, {'message_body': '  reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01t00:02:13z'}, {'message_body': '  why the drop? i warren buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01t00:03:10z'}, {'message_body': '  bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=ba', 'sentiment': -2, 'timestamp': '2018-07-01t00:04:09z'}, {'message

In [18]:
#find username

#regex = re.compile(r'([\w\.-]+)://([\w\.-]+)')
regex = re.compile(r'\@([\w\.-]+)')
#regex = re.compile(r'([\w\.-]+)\$([\w\.-]+)')
matches = regex.finditer(text)

for match in matches:
    print (match)

<_sre.SRE_Match object; span=(129, 140), match='@stocktwits'>


In [19]:
#regex = re.compile(r'\@([\w\.-]+)')
#regex = re.compile(r'([\w\.-]+)\$([\w\.-]+)')

regex = re.compile(r'([a-zA-Z]+)')


matches = regex.finditer(text)

for match in matches:
    print (match)

<_sre.SRE_Match object; span=(3, 10), match='message'>
<_sre.SRE_Match object; span=(11, 15), match='body'>
<_sre.SRE_Match object; span=(20, 24), match='fitb'>
<_sre.SRE_Match object; span=(25, 30), match='great'>
<_sre.SRE_Match object; span=(31, 34), match='buy'>
<_sre.SRE_Match object; span=(35, 37), match='at'>
<_sre.SRE_Match object; span=(46, 49), match='ill'>
<_sre.SRE_Match object; span=(50, 54), match='wait'>
<_sre.SRE_Match object; span=(58, 67), match='sentiment'>
<_sre.SRE_Match object; span=(74, 83), match='timestamp'>
<_sre.SRE_Match object; span=(97, 98), match='t'>
<_sre.SRE_Match object; span=(106, 107), match='z'>
<_sre.SRE_Match object; span=(113, 120), match='message'>
<_sre.SRE_Match object; span=(121, 125), match='body'>
<_sre.SRE_Match object; span=(130, 140), match='stocktwits'>
<_sre.SRE_Match object; span=(142, 146), match='msft'>
<_sre.SRE_Match object; span=(150, 159), match='sentiment'>
<_sre.SRE_Match object; span=(166, 175), match='timestamp'>
<_sre.SRE_

In [20]:
remove_dig = re.sub(r'\d+', " ", text)

remove_dig

               

"[{'message_body': '$fitb great buy at  . ...ill wait', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '@stocktwits $msft', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '#staanalystalert for $tdg : jefferies maintains with a rating of hold setting target price at usd  . . our own verdict is buy  http://www.stocktargetadvisor.com/toprating', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '$amd i heard there’s a guy who knows someone who thinks somebody knows something - on stocktwits.', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '$amd reveal yourself!', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '$aapl why the drop? i warren buffet taking out his position?', 'sentiment':  , 'timestamp': ' - - t : : z'}, {'message_body': '$ba bears have   reason on  -  to pay more attention https://dividendbot.com?s=ba', 'sentiment': - , 'timestamp': ' - - t : : z'}, {'message_body': '$bac ok good we&# ;re not dropp

In [21]:
#remove_char = re.sub(r'[^A-Za-z0-9]+', " ", text)
remove_char = re.sub(r'[^a-zA-Z]+', " ", remove_dig)

remove_char

' message body fitb great buy at ill wait sentiment timestamp t z message body stocktwits msft sentiment timestamp t z message body staanalystalert for tdg jefferies maintains with a rating of hold setting target price at usd our own verdict is buy http www stocktargetadvisor com toprating sentiment timestamp t z message body amd i heard there s a guy who knows someone who thinks somebody knows something on stocktwits sentiment timestamp t z message body amd reveal yourself sentiment timestamp t z message body aapl why the drop i warren buffet taking out his position sentiment timestamp t z message body ba bears have reason on to pay more attention https dividendbot com s ba sentiment timestamp t z message body bac ok good we re not dropping in price over the weekend lol sentiment timestamp t z message body amat daily chart we need to get back to above sentiment timestamp t z message body gme drop per week after spike if no news in months back to s if bo then bingo what is the odds sen

In [22]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    #text = re.sub(r'([\w\.-]+)://([\w\.-]+)', " ", text) #ar!
    
    site_pattern = r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+' #**
    text = re.sub(site_pattern, ' ', text) #**
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    #text = re.sub(r'\$([\w\.-]+)', " ", text) #ar!
    
    ticker_pattern = r'\$\w*\b' #**
    text = re.sub(ticker_pattern, ' ', text) #**
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    #text = re.sub(r'\@([\w\.-]+)', " ", text) #ar!
    
    user_pattern = r'\@\w*\b' #**
    text = re.sub(user_pattern, ' ', text) #**

    # Replace everything not a letter with a space
    #text = re.sub(r'\d+', " ", text) #ar!
    #text = re.sub(r'[^a-zA-Z]+', " ", text) #ar!
    
    not_letter_pattern = r'[^A-Za-z]' #**
    text = re.sub(not_letter_pattern, ' ', text) #**
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    
    #tokens = [word for word in tokens if len(word) >= 2] #ar!
    #tokens = [wnl.lemmatize(w) for w in tokens] #ar!
    
    tokens = [wnl.lemmatize(token,pos='n') for token in tokens if len(token) > 1] #**
    tokens = [wnl.lemmatize(token,pos='v') for token in tokens if len(token) > 1] #**
    
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [23]:
import nltk

In [24]:
from nltk import word_tokenize

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
messages

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option straddle near the supporting

In [27]:

#tokenized = preprocess(str(messages))

"""
tokenized = []

for m in tqdm(str(messages)):
    tokenized.append(preprocess(m))

def word_tokenize(m):
    tokenized = [ w for w in word_tokenizer.tokenize(m)]
    return tokenized


#tokenized = for w in tqdm(word_tokenizer.tokenize(messages))

#tokenized

#tokenized = [word_tokenize(i) for i in messages]

#for i in tokenized:
  #  print (i)
  
from nltk.tokenize import sent_tokenize

tokenized = sent_tokenize(str(messages))

print(tokenized)
"""
tokenized = [preprocess(message) for message in messages]


In [28]:
tokenized

[['great', 'buy', 'at', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'of',
  'hold',
  'set',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'be',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'the', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention'],
 ['ok',
  'good',
  'we',
  're',
  'not',
  'drop',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'if',
  'no',
  'news',
  'in',
  'month',
  'back',
  'to',
  'if',
  'bo',
  'then',
  'what',
  'be',
  'the',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'be', 'at', 'and', 'short', 'to', 'float', 'be', 'via'],


In [29]:
#tokenized(str(messages))

In [30]:
# TODO Implement

"""    
tokenized = preprocess(str(twits['data']))
tokenized
"""

"    \ntokenized = preprocess(str(twits['data']))\ntokenized\n"

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [31]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

#bow = Counter(tokenized).items()
"""
from tqdm import tqdm

bow = Counter()

for tokens in tqdm(tokenized):
    bow.update(tokens)

""" 
#** below    

word_list = []
for twit in tokenized:
    for word in twit:
        word_list.append(word)

bow = Counter(word_list)


In [32]:
bow

Counter({'great': 17305,
         'buy': 117871,
         'at': 137954,
         'wait': 18429,
         'staanalystalert': 4072,
         'for': 273380,
         'jefferies': 1061,
         'maintain': 1852,
         'with': 68768,
         'rat': 16012,
         'of': 211344,
         'hold': 39186,
         'set': 9688,
         'target': 16838,
         'price': 39594,
         'usd': 2926,
         'our': 12188,
         'own': 10907,
         'verdict': 3736,
         'be': 525766,
         'hear': 3798,
         'there': 25284,
         'guy': 12993,
         'who': 17739,
         'know': 22369,
         'someone': 5151,
         'think': 34034,
         'somebody': 1094,
         'something': 5825,
         'on': 241250,
         'stocktwits': 69733,
         'reveal': 482,
         'yourself': 1175,
         'why': 21449,
         'the': 398262,
         'drop': 22681,
         'warren': 1011,
         'buffet': 1280,
         'take': 30007,
         'out': 55926,
         'h

In [33]:
#bow #old

In [34]:
""" ar!
bow = Counter()

for tokens in tokenized:
    bow.update(tokens)
bow
"""

' ar!\nbow = Counter()\n\nfor tokens in tokenized:\n    bow.update(tokens)\nbow\n'

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [35]:
"""
total = sum(tokenized.values(), 0.0)
freq = {k: v / total for k, v in tokenized.items()}
freq
"""

#b1 = bow.items() #ar!

#b1 #ar!

'\ntotal = sum(tokenized.values(), 0.0)\nfreq = {k: v / total for k, v in tokenized.items()}\nfreq\n'

In [36]:
"""
freqs = dict(bow/len(tokenized))
freqs
"""
#freqs = dict((x, y/len(tokenized)) for x, y in b1) #ar!

#freqs #ar!


'\nfreqs = dict(bow/len(tokenized))\nfreqs\n'

In [37]:
#v = bow.count()

In [38]:
#bow = bow/len(tokenized)
"""
total = len(tokenized)

v = dict(bow)

total_v = sum(v.itervalues(), 0.0)

freq = {k: total_v / total for k, v in a.iteritems()}


for i in v.values():
    freq = i / total
    
#freq = {k: v / total for k, v in total}

freq
"""

'\ntotal = len(tokenized)\n\nv = dict(bow)\n\ntotal_v = sum(v.itervalues(), 0.0)\n\nfreq = {k: total_v / total for k, v in a.iteritems()}\n\n\nfor i in v.values():\n    freq = i / total\n    \n#freq = {k: v / total for k, v in total}\n\nfreq\n'

In [39]:
sorted_tokens = sorted(bow.items(), key=lambda pair: pair[1], reverse=True)[:10]
sorted_tokens

[('be', 525766),
 ('the', 398262),
 ('amp', 391858),
 ('to', 379762),
 ('for', 273380),
 ('utm', 271341),
 ('on', 241250),
 ('of', 211344),
 ('and', 207656),
 ('in', 204598)]

In [41]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement STILL to DO!!!

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
#freqs = dict(bow) / len(tokenized)# still to do!!!

#tot_messages = len(tokenized) #** nearly same as own code but worth testing
tot_messages = len(tokenized) #**
freqs = {word:word_count/tot_messages for word,word_count in bow.items()} #**

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000005

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
#sorted_tokens = sorted(bow.items(), key=lambda pair: pair[1], reverse=True)[:k]
#sorted_tokens = sorted(tokenized, key=lambda x: x[1], reverse=True)[k]

k = high_cutoff
#K_most_common = sorted_tokens[:high_cutoff]
#K_most_common = sorted(bow.items(), key=lambda pair: pair[1], reverse=True)[:k] #ar!

K_most_common_list = bow.most_common(high_cutoff) #**
K_most_common = [K_most_common_list[ii][0] for ii in range(high_cutoff)] #**

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['be', 'the', 'amp', 'to', 'for', 'utm', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up']


18447

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [42]:
#vocab = dict((x, y) for x, y in filtered_words) #enumerate(filtered_words,1)
#vocab
filtered_words

['great',
 'buy',
 'wait',
 'staanalystalert',
 'jefferies',
 'maintain',
 'with',
 'rat',
 'hold',
 'set',
 'target',
 'price',
 'usd',
 'our',
 'own',
 'verdict',
 'hear',
 'there',
 'guy',
 'who',
 'know',
 'someone',
 'think',
 'somebody',
 'something',
 'stocktwits',
 'reveal',
 'yourself',
 'why',
 'drop',
 'warren',
 'buffet',
 'take',
 'out',
 'his',
 'position',
 'bear',
 'have',
 'reason',
 'pay',
 'more',
 'attention',
 'ok',
 'good',
 'we',
 're',
 'not',
 'over',
 'weekend',
 'lol',
 'daily',
 'chart',
 'need',
 'get',
 'back',
 'above',
 'per',
 'week',
 'after',
 'if',
 'no',
 'news',
 'month',
 'bo',
 'then',
 'what',
 'odds',
 'strong',
 'short',
 'ratio',
 'float',
 'via',
 'squeeze',
 'perfect',
 'place',
 'an',
 'option',
 'straddle',
 'near',
 'support',
 'trend',
 'start',
 'new',
 'monday',
 'expect',
 'volume',
 'across',
 'key',
 'company',
 'various',
 'sector',
 'breakout',
 'strategy',
 'current',
 'portfolio',
 'bull',
 'catalyst',
 'continue',
 'uptrend',


In [ ]:
#vocab = dict((x, enumerate(filtered_words,1)) for x in filtered_words)
#vocab

In [43]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
#freqs = dict((x, y/len(tokenized)) for x, y in b1)
#vocab = dict(enumerate(filtered_words, 1))
#vocab = (enumerate(filtered_words, 1))
#vocab = dict((x,enumerate(filtered_words, 1) for x, y in filered_words))
#vocab = dict((x, enumerate(filtered_words,1)) for x in filtered_words)
vocab = {word:i for i,word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
#id2vocab = dict((y,x) for x, y in vocab)
id2vocab = {v: k for k, v in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
#filtered = [w for w in tokenized if w not in vocab]
#filtered = [w for w in tokenized if w in vocab]
#dictOfWords = { i : listOfStr[i] for i in range(0, len(listOfStr) ) }
#t = {i : tokenized[i] for i in range(0, len(tokenized))}

#filtered = {w for w in t if w in vocab}
filtered = [[w for w in words if w in vocab] for words in tokenized]

In [44]:
filtered

[['great', 'buy', 'wait'],
 [],
 ['staanalystalert',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'hold',
  'set',
  'target',
  'price',
  'usd',
  'our',
  'own',
  'verdict',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'pay', 'more', 'attention'],
 ['ok', 'good', 'we', 're', 'not', 'drop', 'price', 'over', 'weekend', 'lol'],
 ['daily', 'chart', 'we', 'need', 'get', 'back', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'if',
  'no',
  'news',
  'month',
  'back',
  'if',
  'bo',
  'then',
  'what',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'short', 'float', 'via'],
 ['price',
  'squeeze',
  'perfect',
  'place',
  'an',
  'option',
  'straddle',
  'near',
  'support',
  'trend'],
 ['start',
  'new',
  'monday',
  'expect',
  'strong',
  'b

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [45]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [46]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19440709031596246

Finally let's convert our tokens into integer ids which we can pass to the network.

In [47]:
balanced['messages']

[['great', 'buy', 'wait'],
 ['staanalystalert',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'hold',
  'set',
  'target',
  'price',
  'usd',
  'our',
  'own',
  'verdict',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position'],
 ['bear', 'have', 'reason', 'pay', 'more', 'attention'],
 ['ok', 'good', 'we', 're', 'not', 'drop', 'price', 'over', 'weekend', 'lol'],
 ['daily', 'chart', 'we', 'need', 'get', 'back', 'above'],
 ['drop',
  'per',
  'week',
  'after',
  'if',
  'no',
  'news',
  'month',
  'back',
  'if',
  'bo',
  'then',
  'what',
  'odds'],
 ['strong', 'buy'],
 ['short', 'ratio', 'short', 'float', 'via'],
 ['price',
  'squeeze',
  'perfect',
  'place',
  'an',
  'option',
  'straddle',
  'near',
  'support',
  'trend'],
 ['start',
  'new',
  'monday',
  'expect',
  'strong',
  'buy',


In [ ]:
#vocab[word]

In [48]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        #self.embedding = nn.Embedding(vocab_size, embed_size) #ar!
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        #self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,
         #                  dropout=dropout, batch_first=True) #ar! remove batch_first=True? But Ricardo suggests batch_first=False*
        
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers,
                            dropout=dropout, batch_first=True) 
        # Setup additional layers
        self.dropout = nn.Dropout(0.3) #nn.Dropout(dropout) *
        
        self.fc = nn.Linear(lstm_size, output_size)
        self.sig = nn.Sigmoid()
        self.sft = nn.Softmax(dim=0) #self.log_sft = nn.LogSoftmax(dim=1) *


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        train_on_gpu = torch.cuda.is_available() #*
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
            
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden #*
        
        #hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
         #             weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        
        #hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
         #             weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
   


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        batch_size = nn_input.size(0)
        
        #embeddings & LSTM
        nn_input= nn_input.long()
        embeds = self.embedding(nn_input)
        
        lstm_out, hidden_state = self.lstm(embeds, hidden)
        
        #stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        #lstm_out[-1,:,:]
        
        #dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        #softmax function instead of sigmoid
        sft_out = self.sft(out)
        #logps = self.log_sft(out) *
        
        #reshape to be batch_size first
        sft_out = sft_out.view(batch_size, -1)
        sft_out = sft_out[:, -1] #get last batch labels
        
        #logps = logps.view(batch_size, -1) * Ricardo advised to remove these since lstm_out[-1,:,:]
        #logps = logps[:, -1] #get last batch labels *
        
        
        #return last softmax output and hidden state
        
        return sft_out, hidden_state
        #return logps, hidden_state *
    

In [49]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [50]:
#Amended

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        #self.embedding = nn.Embedding(vocab_size, embed_size) #ar!
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size) #**
        
        #self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,
                           #dropout=dropout, batch_first=True) #remove batch_first=True? But Ricardo suggests batch_first=False*
        
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers,
                           dropout=self.dropout, batch_first=False) #**self.dropout
        # Setup additional layers
        #self.dropout = nn.Dropout(0.3) #nn.Dropout(dropout) *
        #self.dropout = nn.Dropout(dropout)
        self.drop_layer = nn.Dropout(self.dropout) #**
        
        #self.fc = nn.Linear(lstm_size, output_size) #ar!
        self.fc = nn.Linear(self.lstm_size, self.output_size) #**
        
        #self.sig = nn.Sigmoid()
        #self.sft = nn.Softmax(dim=0) #self.log_sft = nn.LogSoftmax(dim=1) *
        #self.log_sft = nn.LogSoftmax(dim=1) #ar!
        self.log_softmax = nn.LogSoftmax(dim=1) #**

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        #train_on_gpu = torch.cuda.is_available() #*
        
        weight = next(self.parameters()).data
        """
        if (train_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
            
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden #*
        """
       
        #hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
         #             weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden
    
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        #batch_size = nn_input.size(0) #** suggests removing this
        
        #embeddings & LSTM
        nn_input= nn_input.long()
        
        embeds = self.embedding(nn_input)
        
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        
        #stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        
        lstm_out = lstm_out[-1,:,:] #*
        
        #dropout and fully-connected layer
        #out = self.dropout(lstm_out) #**out = self.drop_layer(lstm_out) need to change layer above if so
        out = self.drop_layer(lstm_out) #**
        out = self.fc(out)
        
        #softmax function instead of sigmoid
        #sft_out = self.sft(out)
        #logps = self.log_sft(out) #*
        logps = self.log_softmax(out) #**
        
        #reshape to be batch_size first
        #sft_out = sft_out.view(batch_size, -1)
        #sft_out = sft_out[:, -1] #get last batch labels
        
        #logps = logps.view(batch_size, -1) #* Ricardo advised to remove these since lstm_out[-1,:,:]
        #logps = logps[:, -1] #get last batch labels *
        
        
        #return last softmax output and hidden state
        
        #return sft_out, hidden_state
        return logps, hidden_state #*
    

### View Model

In [51]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
#input = input.to('cuda')
#hidden = model.init_hidden(4)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.2111, -1.8594, -2.0251, -1.6563, -1.4981],
        [-1.2068, -1.8397, -2.0184, -1.6724, -1.5081],
        [-1.2077, -1.7961, -2.0529, -1.7023, -1.4937],
        [-1.2367, -1.7635, -2.0661, -1.6988, -1.4755]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [52]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [53]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8

split_idx = int(len(token_ids)*split_frac)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]


In [54]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(18448, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (drop_layer): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [ ]:
model

In [57]:
""" TRAINING MODEL WORX!
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 512
learning_rate = 0.002

#seq_length = 20

print_every = 100
clip =5 #ar! gradient clipping
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    #hidden = model.init_hidden(batch_size) #ar! this is covered above in dataloader specfies batch size
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        #hidden = pass
        
        #hidden = model.init_hidden(labels.size[0]) #following example
        hidden = model.init_hidden(labels.shape[0])
        print(hidden)
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        hidden = tuple([each.data for each in hidden]) #ar!
        
        #zero accumulated gradients ar!
        model.zero_grad() 
        
        #get the output from the model ar!
        output, hidden = model(text_batch, hidden) #model.forward?
        
        #calculate loss and preform backprop ar!
        #loss = criterion(output.sqeeze(), labels.float()) #(output.squeeze(), labels)
        #loss = criterion(output.sqeeze(), labels)
        #loss = criterion(output, labels) #followning example
        loss = criterion(output, labels.long()) #**
        loss.backward()
        
        #clip_grad_orm prevents exploding rnn ar!
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        
        # TODO Implement: Train Model
        
        
        if steps % print_every == 0:
            #Get validatin loss ar!
            #val_h = net.init_hidden(batch_size) #ar!
            #val_h = net.init_hidden(labels.shape[0])
            #val_losses = [] #ar!
            val_losses = [] #**
            
            model.eval()
            
            #val_h = model.init_hidden(labels.shape[0]) #ar!
            #val_h = model.init_hidden(batch_size)
            #val_losses = [] #ar!
            #accuracy = []
            
            # TODO Implement: Print metrics
            #for text_batch, labels in valid_loader: ar! not relevant as no validation
           
            #for text_batch, labels in dataloader(
             #   valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
                
            for val_batch, val_labels in dataloader(
                 valid_features, valid_labels, batch_size=batch_size, sequence_length=20, shuffle=True): #**
                
                #val_h = model.init_hidden(labels.shape[0])
                val_h = model.init_hidden(val_labels.shape[0]) #**
                
                val_h = tuple([each.data for each in val_h]) #**
                
                val_batch, val_labels = text_batch.to(device), labels.to(device) #**
                
                for each in val_h:
                    each.to(device)
                    #val_h = model.init_hidden(labels.shape[0])
                
                #val_h = tuple([each.data for each in val_h]) #ar!
                #text_batch, labels = text_batch.to(device), labels.to(device) *ar!
                #output, val_h = model(inputs, val_h)
                #output, val_h = model(text_batch, val_h) #ar!
                
                #output, val_h = model(text_batch, val_h) #ar!
                
                val_out, val_h = model.forward(val_batch, val_h) #**
                
                #val_loss = criterion(output.squeeze(), labels.float()) *ar!
                val_loss = criterion(val_out, val_labels.long()) #**
                
                val_losses.append(val_loss.item())
                
                #calculate metrics **
                ps = torch.exp(val_out)
                top_p, top_class = ps.topk(1,dim=1)
            
            model.train()
                        
            """
            print("Epoch: {}/{}...".format(e+1, epochs),
                 "Step: {}...".format(steps),
                 "Loss: {:.6f}...".format(loss,item()))
            """
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                 "Step: {}...".format(steps),
                 "Loss: {:.6f}...".format(loss,item()),
                 "Val Loss: {:.6f}".format(np.mean(val_losses)))
            

Starting epoch 1
(tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]], device='cuda:0'), tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.

KeyboardInterrupt: 

In [ ]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 1
batch_size = 50
learning_rate = 0.001

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = pass
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            
            model.train()

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [ ]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    #vocab = {word:i for i,word in enumerate(filtered_words, 1)}
    #id2vocab = {v: k for k, v in vocab.items()}
    #filtered = [[w for w in words if w in vocab] for words in tokenized]
    
    # Filter non-vocab words
    tokens = [[w for w in words if w not in vocab] for words in tokenized] #ar!
    # Convert words to ids
    tokens = {word:i for i,word in enumerate(tokens, 1)} #ar!
        
    # Adding a batch dimension
    text_input = feature_tesnor.size()
    
    # Get the NN output
    hidden = model.init_hidden(text_input)
    #logps, _ = pass
    logps, hidden = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.round(logps.squeeze())
    
    return pred

In [ ]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [ ]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [ ]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

Using the `prediction` function, let's apply it to a stream of twits.

In [ ]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [ ]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.